In [1]:
# ============================================
# Cell 0 — Experiment plan & seeds (GLOBAL)
# ============================================
# We'll run 5 instances (same across all notebooks) and two depths: 1 and 3 layers.
INSTANCE_IDS   = [1, 2, 3, 4, 5]   # used in filenames as ..._ls_01.json, ..._ls_02.json, ...
LAYER_OPTIONS  = [1, 3]            # train 1-layer first, then 3-layers
EVAL_SIGMA     = 0.2             # fixed noise everywhere (train & eval)

# where to save artifacts (JSON bundles, instance records, CSV summary)
OUT_BASE = "./runs_halfqae"        # change if you like; subfolders will be created automatically
CSV_PATH = f"{OUT_BASE}/results_instances.csv"  # will be appended-to if exists

In [2]:
# =====================================================
# Cell 1 — Imports, utils, reproducibility (fixed seed)
# =====================================================
import os, sys, json, math, random, time, hashlib
from pathlib import Path
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# ----- repo utils (your existing readers) -----
current_dir = os.path.dirname(os.path.abspath(''))
parent_dir = os.path.dirname(current_dir) if os.path.basename(current_dir) == 'Jacob' else current_dir
sys.path.insert(0, parent_dir); sys.path.insert(0, '../')
try:
    from qae_utils.Files import read_ts_file, read_json_file
    print("Utils import OK")
except Exception as e:
    print("Import error:", e)
    qae_utils_path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'qae_utils')
    sys.path.insert(0, os.path.dirname(qae_utils_path))
    from qae_utils.Files import read_ts_file, read_json_file
    print("Absolute path fallback OK")

assert callable(read_ts_file) and callable(read_json_file)

# ----- plotting defaults -----
np.set_printoptions(suppress=True, precision=6)
plt.rcParams["figure.figsize"] = (6.5, 4)

# ----- reproducibility -----
def set_global_seed(instance_id: int):
    """
    Derive all RNGs from a simple instance ID (1..5).
    Keep the mapping stable across notebooks.
    """
    base = 10_000 + int(instance_id)  # simple, memorable
    random.seed(base + 11)
    np.random.seed(base + 22)
    try:
        pnp.random.seed(base + 33)
    except Exception:
        pass
    # Pennylane's default.qubit is deterministic given params; no device seeding needed.
    return dict(global_seed=base, numpy_seed=base+22, pnp_seed=base+33)

def std_instance_name(nq, n_latent, n_trash, n_layers, instance_id):
    """
    Standardized filename pattern used across the project.
    Example: 4q_2l_2t_3ls_01.json
    """
    return f"{int(nq)}q_{int(n_latent)}l_{int(n_trash)}t_{int(n_layers)}ls_{int(instance_id):02d}.json"

def ensure_dir(p): Path(p).mkdir(parents=True, exist_ok=True); return p

print("Seed/filename utils ready.")

Utils import OK
Seed/filename utils ready.


In [3]:
# =========================================
# Cell 2 — Data loading (deterministic)
# =========================================
DATA_PATH = '../jacobs_examples/aintern/data'
DATA_NAME = 'mackey_glass'  # your folder prefix

# fixed split across all instances/layers (so results are comparable)
SPLIT_RANDOM_STATE = 42
WINDOW_STRIDE = 1

# ---- pick most recent MG folder starting with mackey_glass* ----
data_folders = [f for f in os.listdir(DATA_PATH) if f.startswith(DATA_NAME)]
if not data_folders:
    raise FileNotFoundError("No Mackey-Glass data found. Generate it first.")
data_folders.sort()
data_folder = data_folders[-1]     # take the last one if multiple
print(f"Using data folder: {data_folder}")

# ---- load series + scaling info ----
X_idx = read_ts_file(f'{DATA_PATH}/{data_folder}/x_org.arr')   # indices for plotting, not used
y_all = read_ts_file(f'{DATA_PATH}/{data_folder}/y_org.arr')   # values
info  = read_json_file(f'{DATA_PATH}/{data_folder}/info.json')

print(f"Loaded {len(y_all)} samples; scale [{info['scale_low']:.3f},{info['scale_high']:.3f}]")

# ---- helper: uniform embed wrapper (works with/without explicit info param)
def embed_input(x, info_=None):
    """
    Map value-domain window x (in [lo,hi]) to RY(π·v01).
    Accepts optional info to match Stage-3 call signatures.
    """
    if info_ is None:
        info_ = info
    lo, hi = info_['scale_low'], info_['scale_high']
    xn = (pnp.array(x) - lo) / max(hi - lo, 1e-12)   # -> [0,1]
    for i, v in enumerate(xn):
        qml.RY(v * pnp.pi, wires=i)

Using data folder: mackey_glass_tau30_n200
Loaded 200 samples; scale [0.200,0.800]


In [4]:
# ============================================================
# Cell 3 — Architecture (do NOT change your brick/entanglers)
# ============================================================
# This notebook = 4 qubits, 2 latent, 2 trash.
n_qubits = 4
n_latent = 2
n_trash  = n_qubits - n_latent
trash_wires = [2, 3]            # your original choice
signal_wires = list(range(4))   # Stage-1 diagnostics use all wires

# --- device factory (simple; default.qubit) ---
def make_device(nq): 
    return qml.device('default.qubit', wires=nq)

# --- Stage-1 encoder template (unchanged architecture) ---
def encoder_template(params, n_layers):
    """RX/RY/RZ per qubit + ring CNOT per layer."""
    assert len(params) == n_layers * n_qubits * 3
    for l in range(n_layers):
        # local rotations
        for q in range(n_qubits):
            idx = l * n_qubits * 3 + q * 3
            qml.RX(params[idx + 0], wires=q)
            qml.RY(params[idx + 1], wires=q)
            qml.RZ(params[idx + 2], wires=q)
        # ring entanglers
        for q in range(n_qubits-1):
            qml.CNOT(wires=[q, q+1])
        qml.CNOT(wires=[n_qubits-1, 0])

print("Architecture set: 4q (2 latent and 2 trash).")

Architecture set: 4q (2 latent and 2 trash).


In [5]:
# ====================================================
# Cell 4 — Stage-1 QNodes, loss, and training (seeded)
# ====================================================
def stage1_qnodes(n_layers):
    dev = make_device(n_qubits)

    @qml.qnode(dev, interface="autograd", diff_method="backprop")
    def trash_expectations(phi, x_clean):
        embed_input(x_clean)
        encoder_template(phi, n_layers)
        return [qml.expval(qml.PauliZ(w)) for w in trash_wires]

    @qml.qnode(dev, interface="autograd", diff_method="backprop")
    def recon_EdagE(phi, x_clean):
        embed_input(x_clean)
        encoder_template(phi, n_layers)
        qml.adjoint(encoder_template)(phi, n_layers)   # E†
        return [qml.expval(qml.PauliZ(w)) for w in range(n_qubits)]

    return trash_expectations, recon_EdagE

def stage1_batch_loss(trash_expectations, phi, clean_batch):
    # L = mean_{batch,trash} P(|1>) = (1 - Z)/2
    zs = []
    for c in clean_batch:
        z = pnp.array(trash_expectations(phi, c))  # shape (n_trash,)
        zs.append(z)
    zs = pnp.stack(zs, axis=0)
    prob_one = (1.0 - zs) * 0.5
    return pnp.mean(prob_one)

def train_stage1(X_train, X_val, n_layers, instance_id, 
                 n_epochs=120, batch_size=32, lr_init=0.010,
                 patience=10, lr_patience=8, min_delta=1e-6):
    set_global_seed(instance_id)
    # init
    enc_shape = n_layers * n_qubits * 3
    phi = pnp.array(np.random.normal(0, 0.5, enc_shape), requires_grad=True)
    opt = qml.AdamOptimizer(stepsize=lr_init)
    lr = lr_init

    trash_expectations, recon_EdagE = stage1_qnodes(n_layers)

    # helper
    def minibatches(N, B, rng_seed=123456):
        rng = np.random.default_rng(rng_seed)  # fixed per-epoch seed below
        idx = rng.permutation(N)
        for i in range(0, N, B):
            yield idx[i:i+B]

    train_hist, val_hist, lr_hist = [], [], []
    best_phi, best_val = None, float("inf")
    no_improve = 0
    for ep in range(n_epochs):
        # batch order deterministic per-epoch per-instance
        seed_ep = 77_000 + 100*instance_id + ep
        acc = 0.0; nb = 0
        for ix in minibatches(len(X_train), batch_size, rng_seed=seed_ep):
            clean_batch = X_train[ix]
            def loss_fn(p): return stage1_batch_loss(trash_expectations, p, clean_batch)
            phi, cost = opt.step_and_cost(loss_fn, phi)
            acc += float(cost); nb += 1
        train_cost = acc / max(nb, 1)

        # validation
        v_costs = []
        for c in X_val:
            v_costs.append(stage1_batch_loss(trash_expectations, phi, pnp.array([c])))
        val_cost = float(pnp.mean(pnp.stack(v_costs)))

        train_hist.append(train_cost); val_hist.append(val_cost); lr_hist.append(lr)

        if val_cost + min_delta < best_val:
            best_val, best_phi = val_cost, pnp.array(phi, requires_grad=False); no_improve = 0
        else:
            no_improve += 1
            if (no_improve % lr_patience) == 0:
                lr = max(lr * 0.5, 1e-4)
                opt = qml.AdamOptimizer(stepsize=lr)
                print(f"[Stage1] ↓ LR → {lr:.5f}")
            if no_improve >= patience:
                print("[Stage1] Early stopping."); break

        print(f"[Stage1] L={n_layers} ep {ep:03d} | train {train_cost:.6f} | val {val_cost:.6f} | LR {lr:.5f}")

    phi_best = best_phi if best_phi is not None else phi
    return dict(
        phi=phi_best, best_val=float(best_val),
        hist_train=list(map(float, train_hist)),
        hist_val=list(map(float, val_hist)),
        hist_lr=list(map(float, lr_hist)),
        recon_EdagE=recon_EdagE
    )

In [6]:
# ======================================================================
# Cell 5 — Stage-3 (ψ) with fixed decoder = adjoint(Stage-1 encoder φ)
#          (records hist_noisy, hist_delta, best_epoch, epochs, seconds)
# ======================================================================
import time

def stage3_qnodes(n_layers, phi_stage1):
    dev3 = make_device(n_qubits)
    theta_fixed = pnp.array(phi_stage1, requires_grad=False).reshape((n_layers, n_qubits, 3))

    def encoder_fixed_body(theta):
        for l in range(n_layers):
            for q in range(n_qubits):
                qml.RX(theta[l, q, 0], wires=q)
                qml.RY(theta[l, q, 1], wires=q)
                qml.RZ(theta[l, q, 2], wires=q)
            for q in range(n_qubits-1):
                qml.CNOT(wires=[q, q+1])
            qml.CNOT(wires=[n_qubits-1, 0])

    def decoder_fixed():
        qml.adjoint(encoder_fixed_body)(theta_fixed)

    @qml.qnode(dev3, interface="autograd", diff_method="backprop")
    def encoder_only_expZ_all(flat_params, x_in):
        embed_input(x_in)
        encoder_template(flat_params, n_layers)
        return [qml.expval(qml.PauliZ(w)) for w in range(n_qubits)]

    @qml.qnode(dev3, interface="autograd", diff_method="backprop")
    def teacher_code_latents(x_in):
        embed_input(x_in)
        encoder_fixed_body(theta_fixed)
        return [qml.expval(qml.PauliZ(w)) for w in range(n_latent)]

    @qml.qnode(dev3, interface="autograd", diff_method="backprop")
    def student_code_latents(flat_params, x_in):
        embed_input(x_in)
        encoder_template(flat_params, n_layers)
        return [qml.expval(qml.PauliZ(w)) for w in range(n_latent)]

    @qml.qnode(dev3, interface="autograd", diff_method="backprop")
    def denoiser_qnode_all(flat_params, x_noisy):
        embed_input(x_noisy)
        encoder_template(flat_params, n_layers)
        decoder_fixed()
        return [qml.expval(qml.PauliZ(w)) for w in range(n_qubits)]

    return dict(
        theta_fixed=theta_fixed,
        encoder_only_expZ_all=encoder_only_expZ_all,
        teacher_code_latents=teacher_code_latents,
        student_code_latents=student_code_latents,
        denoiser_qnode_all=denoiser_qnode_all
    )

# ----- value readout helpers (unchanged)
def Z_to_values_autograd(z_all):
    z_all = pnp.clip(pnp.asarray(z_all), -0.999999, 0.999999)
    v01 = pnp.arccos(z_all) / pnp.pi
    return v01 * (info["scale_high"] - info["scale_low"]) + info["scale_low"]

def first_diff(x): 
    x = pnp.array(x); return x[1:] - x[:-1]

def p1_from_expZ(z): 
    return (1 - pnp.asarray(z)) * 0.5

# ----- deterministic noisy window (shared with eval)
def ts_add_noise_window_det(x, sigma, seed):
    low, high = float(info["scale_low"]), float(info["scale_high"])
    rng = np.random.default_rng(int(seed))
    noise = rng.normal(0.0, sigma * (high - low), size=np.asarray(x).shape)
    return np.clip(np.asarray(x) + noise, low, high)

# ----- validation with fixed per-window seeds (unchanged)
VAL_BASE_SEED = 12345

def stage3_val_values_det(psi, X_clean, sigma=EVAL_SIGMA):
    ms_noisy, ms_deno = [], []
    for i, c in enumerate(X_clean):
        n = ts_add_noise_window_det(c, sigma, seed=VAL_BASE_SEED + i)
        zD = np.array(stage3_handles["denoiser_qnode_all"](psi, n))
        v_hat = np.array(Z_to_values_autograd(zD))
        ms_noisy.append(np.mean((np.asarray(c) - np.asarray(n))**2))
        ms_deno.append(np.mean((np.asarray(c) - v_hat)**2))
    mN, mD = float(np.mean(ms_noisy)), float(np.mean(ms_deno))
    d_pct = 100.0 * (1.0 - mD / max(mN, 1e-12))
    return mN, mD, d_pct

# ----- small Huber
def huber(residual, delta):
    r = pnp.abs(residual)
    return pnp.where(r <= delta, 0.5*r**2, delta*(r - 0.5*delta))


def train_stage3(X_train, X_val, phi_stage1, n_layers, instance_id,
                 TARGET_NOISE=EVAL_SIGMA, MAX_EPOCHS=60, BATCH=16, 
                 LR_START=0.003, PATIENCE=10, PLATEAU_STEPS=5, PLATEAU_FACTOR=0.5,
                 CLIP_NORM=2.0, USE_EMA=True, EMA_DECAY=0.99):

    # ---- seeds: varied but reproducible across (instance, layers, epoch, window)
    def make_train_seed(instance_id, layers, ep, k, view=0):
        return (1_000_003 * (instance_id * 10 + layers) + 97 * ep + 31 * int(k) + view) % 2_147_483_647

    set_global_seed(instance_id)

    global stage3_handles
    stage3_handles = stage3_qnodes(n_layers, phi_stage1)
    enc_all = stage3_handles["encoder_only_expZ_all"]
    teacher_lat = stage3_handles["teacher_code_latents"]
    denoise_all = stage3_handles["denoiser_qnode_all"]

    # ---- init ψ near φ
    phi_flat = pnp.array(phi_stage1, requires_grad=False)
    psi = pnp.array(np.array(phi_flat) + 0.05*np.random.randn(len(phi_flat)), requires_grad=True)

    # ---- loss weights
    ALPHA_REC, BETA_TF, GAMMA_TRASH, L_TV, L_ANCH = 1.0, 0.05, 0.5, 0.05, 2e-4
    DELTA_TV, DELTA_Z = 0.02, 0.25

    # loss on a single window with a specific noise seed
    def loss_on_window_seeded(params, clean_values, seed):
        v_noisy = pnp.array(ts_add_noise_window_det(clean_values, TARGET_NOISE, seed=seed))
        z_all = pnp.array(enc_all(params, v_noisy))
        z_sig, z_tr = z_all[:n_latent], z_all[n_latent:]
        zD = pnp.array(denoise_all(params, v_noisy))
        v_hat = Z_to_values_autograd(zD)

        L_rec = pnp.mean((pnp.array(clean_values) - v_hat)**2)
        z_t_sig = pnp.array(teacher_lat(clean_values))
        L_tf = pnp.mean(huber(z_t_sig - z_sig, DELTA_Z))
        L_tr = pnp.mean(p1_from_expZ(z_tr))
        L_tv = pnp.mean(huber(first_diff(clean_values) - first_diff(v_hat), DELTA_TV))
        L_anchor = pnp.mean((params - phi_flat)**2)
        return (ALPHA_REC*L_rec + BETA_TF*L_tf + GAMMA_TRASH*L_tr + L_TV*L_tv + L_ANCH*L_anchor)

    # manual Adam
    m = pnp.zeros_like(psi); v = pnp.zeros_like(psi)
    b1, b2, eps = 0.9, 0.999, 1e-8
    t = 0
    def adam_step(params, grad, lr):
        nonlocal m, v, t
        t += 1
        m = b1*m + (1-b1)*grad
        v = b2*v + (1-b2)*(grad*grad)
        mhat = m/(1-b1**t); vhat = v/(1-b2**t)
        return params - lr * (mhat/(pnp.sqrt(vhat)+eps))

    # batches deterministic per-epoch
    def batch_indices(N, B, ep_seed):
        rng = np.random.default_rng(ep_seed)
        idx = rng.permutation(N)
        for s in range(0, N, B):
            yield idx[s:s+B]

    best_params, best_val = None, float("inf")
    best_epoch = -1
    no_improve, lr = 0, LR_START
    ema = pnp.array(psi, requires_grad=False) if USE_EMA else None

    # history buffers (for CSV/reporting)
    hist_train, hist_val = [], []
    hist_noisy, hist_delta = [], []

    t0 = time.time()

    for ep in range(MAX_EPOCHS):
        seed_ep = 88_000 + 100*instance_id + ep  # reproducible shuffling
        acc, nb = 0.0, 0
        for ix in batch_indices(len(X_train), BATCH, ep_seed=seed_ep):
            for k in ix:                                   # k = absolute index in X_train
                c = X_train[k]
                seed = make_train_seed(instance_id, n_layers, ep, int(k))
                L = loss_on_window_seeded(psi, c, seed)
                if not pnp.isfinite(L): 
                    continue
                g = qml.grad(lambda p: loss_on_window_seeded(p, c, seed))(psi)
                if not pnp.all(pnp.isfinite(g)): 
                    continue
                # clip
                gnorm = pnp.linalg.norm(g) + 1e-12
                if gnorm > CLIP_NORM:
                    g = g * (CLIP_NORM / gnorm)
                psi = adam_step(psi, g, lr)
                if USE_EMA: 
                    ema = EMA_DECAY*ema + (1-EMA_DECAY)*psi
                acc += float(L); nb += 1

        train_loss = acc / max(nb, 1)
        eval_params = ema if USE_EMA else psi

        # strict value-domain validation at σ=EVAL_SIGMA (deterministic per window)
        mN, mD, dV = stage3_val_values_det(eval_params, X_val, sigma=EVAL_SIGMA)
        hist_train.append(train_loss); hist_val.append(mD)
        hist_noisy.append(mN);        hist_delta.append(dV)

        if mD < best_val - 1e-12:
            best_val, best_params, best_epoch, no_improve = mD, pnp.array(eval_params, requires_grad=False), ep, 0
        else:
            no_improve += 1
            if (no_improve % PLATEAU_STEPS) == 0:
                lr *= PLATEAU_FACTOR
                print(f"[Stage3] Plateau → LR {lr:.5f}")

        norm_diff = float(pnp.linalg.norm((eval_params - phi_flat)))
        print(f"[Stage3] L={n_layers} ep {ep:03d} | train {train_loss:.5f} | "
              f"val {mD:.5f} | noisy {mN:.5f} | Δ {dV:+.1f}% | LR {lr:.5f} | ||ψ-φ|| {norm_diff:.3f}")

        if no_improve >= PATIENCE:
            print("[Stage3] Early stopping."); break

    train_seconds = float(time.time() - t0)
    epochs_run = len(hist_val)

    psi_best = best_params if best_params is not None else (ema if USE_EMA else psi)

    return dict(
        psi=psi_best, 
        best_val=float(best_val),
        best_epoch=int(best_epoch),
        epochs=int(epochs_run),
        hist_train=list(map(float, hist_train)),
        hist_val=list(map(float, hist_val)),
        hist_noisy=list(map(float, hist_noisy)),
        hist_delta=list(map(float, hist_delta)),
        train_seconds=train_seconds
    )

In [7]:
# ===================================================
# Cell 6 — Build windows & deterministic train/val/test
# ===================================================
window_size = n_qubits
stride = WINDOW_STRIDE

X_windows = np.array([y_all[i:i+window_size] for i in range(0, len(y_all)-window_size+1, stride)], dtype=float)
print(f"Total windows built: {len(X_windows)} (W={window_size}, step={stride})")

# 60/20/20 split (deterministic)
X_temp, X_test = train_test_split(X_windows, test_size=0.20, random_state=SPLIT_RANDOM_STATE)
X_train, X_val = train_test_split(X_temp,   test_size=0.25, random_state=SPLIT_RANDOM_STATE)  # 0.25 of 0.8 = 0.2
print(f"Split sizes → train={len(X_train)}, val={len(X_val)}, test={len(X_test)}")

Total windows built: 197 (W=4, step=1)
Split sizes → train=117, val=40, test=40


In [8]:
# ============================================
# Cell 7 — Train runs (instances × layers)
# ============================================
RUNS = []  # we’ll save each run in the next cell

for L in LAYER_OPTIONS:
    for inst in INSTANCE_IDS:
        print(f"\n==============================")
        print(f"Instance {inst} | Layers {L}")
        print(f"==============================")

        t0 = time.time()
        s1 = train_stage1(
            X_train, X_val,
            n_layers=L,
            instance_id=inst,
            n_epochs=120, batch_size=32,
            lr_init=0.010, patience=10, lr_patience=8, min_delta=1e-6
        )
        t1 = time.time()

        s3 = train_stage3(
            X_train, X_val,
            phi_stage1=s1["phi"],
            n_layers=L,
            instance_id=inst,
            TARGET_NOISE=EVAL_SIGMA, MAX_EPOCHS=60, BATCH=16,
            LR_START=0.003, PATIENCE=10, PLATEAU_STEPS=5, PLATEAU_FACTOR=0.5,
            CLIP_NORM=2.0, USE_EMA=True, EMA_DECAY=0.99
        )
        t2 = time.time()

        RUNS.append({
            "instance_id": inst,
            "n_layers": L,
            "stage1": {
                "phi": s1["phi"],
                "best_val": s1["best_val"],
                "hist_train": s1["hist_train"],
                "hist_val": s1["hist_val"],
                "hist_lr": s1["hist_lr"],
                "best_epoch": s1.get("best_epoch"),
                "epochs": s1.get("epochs"),
                "train_seconds": float(t1 - t0),
            },
            "stage3": {
                "psi": s3["psi"],
                "best_val": s3["best_val"],
                "best_epoch": s3.get("best_epoch"),
                "epochs": s3.get("epochs"),
                "hist_train": s3["hist_train"],
                "hist_val": s3["hist_val"],
                # NEW: capture these so Cell 8 has them
                "hist_noisy": s3.get("hist_noisy", []),
                "hist_delta": s3.get("hist_delta", []),
                "train_seconds": float(t2 - t1),
            }
        })

print(f"\nCompleted {len(RUNS)} runs.")


Instance 1 | Layers 1
[Stage1] L=1 ep 000 | train 0.480592 | val 0.472863 | LR 0.01000
[Stage1] L=1 ep 001 | train 0.475778 | val 0.470310 | LR 0.01000
[Stage1] L=1 ep 002 | train 0.475132 | val 0.467689 | LR 0.01000
[Stage1] L=1 ep 003 | train 0.472342 | val 0.465232 | LR 0.01000
[Stage1] L=1 ep 004 | train 0.468208 | val 0.462826 | LR 0.01000
[Stage1] L=1 ep 005 | train 0.467239 | val 0.460468 | LR 0.01000
[Stage1] L=1 ep 006 | train 0.461400 | val 0.458210 | LR 0.01000
[Stage1] L=1 ep 007 | train 0.458514 | val 0.456005 | LR 0.01000
[Stage1] L=1 ep 008 | train 0.454959 | val 0.453897 | LR 0.01000
[Stage1] L=1 ep 009 | train 0.450856 | val 0.451793 | LR 0.01000
[Stage1] L=1 ep 010 | train 0.446003 | val 0.449721 | LR 0.01000
[Stage1] L=1 ep 011 | train 0.447651 | val 0.447556 | LR 0.01000
[Stage1] L=1 ep 012 | train 0.443245 | val 0.445356 | LR 0.01000
[Stage1] L=1 ep 013 | train 0.437866 | val 0.443128 | LR 0.01000
[Stage1] L=1 ep 014 | train 0.437992 | val 0.440782 | LR 0.01000
[S

In [10]:
# ======================================================================
# Cell 8 — Save artifacts (JSON) and append a paper-ready CSV per run
# ======================================================================
from pathlib import Path
import json, time, os, csv
import numpy as np
import pandas as pd

# --- hyperparams logged (keep in sync with training cells) ---
S1_LR_INIT       = 0.010
S1_MAX_EPOCHS    = 120
S1_PATIENCE      = 10
S1_LR_PATIENCE   = 8

S3_LR_INIT       = 0.003
S3_MAX_EPOCHS    = 60
S3_PATIENCE      = 10
S3_PLATEAU_STEPS = 5
S3_PLATEAU_FACT  = 0.5

CSV_SCHEMA_VERSION = "v2"  # bump if you change columns

# --- ensure dirs ---
ensure_dir(OUT_BASE)
subroot = ensure_dir(f"{OUT_BASE}/q{n_qubits}_l{n_latent}t{n_trash}")

# --- CSV path (versioned) ---
CSV_PATH = f"{OUT_BASE}/all_training_instances_{CSV_SCHEMA_VERSION}.csv"

# --- header for the full, paper-friendly table ---
CSV_HEADER = [
    # id / naming
    "filename","run_tag","dataset_folder","instance_id","rng_seed",
    # architecture
    "n_qubits","n_latent","n_trash","n_layers",
    # noise & window
    "sigma_train","sigma_eval","window_stride",
    # stage-1 hyperparams + outcomes
    "s1_lr_init","s1_max_epochs","s1_patience","s1_lr_patience",
    "s1_best_val","s1_final_val","s1_best_epoch","s1_epochs","s1_train_seconds",
    # stage-3 hyperparams + outcomes
    "s3_lr_init","s3_max_epochs","s3_patience","s3_plateau_steps","s3_plateau_factor",
    "s3_best_val_mse","s3_final_val_mse","s3_best_epoch","s3_epochs","s3_train_seconds",
    "s3_noisy_baseline_mse","s3_best_delta_pct","s3_final_delta_pct",
    # params (JSON)
    "phi_params","psi_params",
    # totals
    "total_train_seconds",
]

def ensure_csv(path, header):
    needs_header = True
    if os.path.exists(path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                first_line = f.readline().rstrip("\n")
            needs_header = (first_line != ",".join(header))
        except Exception:
            needs_header = True
    if needs_header:
        with open(path, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(header)

ensure_csv(CSV_PATH, CSV_HEADER)

# -------------------------
# Robust helper utilities
# -------------------------
def _safe_argmin(seq):
    try:
        return int(np.nanargmin(seq)) if len(seq) else -1
    except Exception:
        return -1

def _safe_last(seq):
    return float(seq[-1]) if (isinstance(seq, (list, tuple)) and len(seq)) else np.nan

def _safe_int(x, default):
    """Cast to int, handling None/NaN/invalid gracefully."""
    try:
        if x is None:
            return int(default)
        if isinstance(x, float) and np.isnan(x):
            return int(default)
        return int(x)
    except Exception:
        return int(default)

def _safe_float(x, default=np.nan):
    try:
        if x is None:
            return float(default)
        return float(x)
    except Exception:
        return float(default)

# Fallback filename formatter if not provided elsewhere
if "std_instance_name" not in globals():
    def std_instance_name(nq, nl, nt, L, inst):
        return f"{nq}q_{nl}l_{nt}t_L{L}_inst{inst:02d}.json"

# -------------------------
# Saver
# -------------------------
def save_one_run(run):
    global stage3_handles  # used if we need to rebuild QNodes for fallback eval

    # Architecture context
    nq, nl, nt = int(n_qubits), int(n_latent), int(n_trash)

    # Try to grab dataset folder/name from globals or default
    dataset_folder = globals().get("data_folder", None)
    if dataset_folder is None:
        dataset_folder = globals().get("dataset_folder", "unknown_dataset")

    # Try to get scaling info; default to [0,1] if missing
    _info = globals().get("info", {}) or {}
    scale_low  = float(_info.get("scale_low", 0.0))
    scale_high = float(_info.get("scale_high", 1.0))

    # Paths & file names
    inst = _safe_int(run.get("instance_id"), 0)
    L    = _safe_int(run.get("n_layers"), 0)
    seed = _safe_int(run.get("seed", inst), inst)

    fname = std_instance_name(nq, nl, nt, L, inst)
    out_dir = ensure_dir(f"{subroot}/L{L}")
    bundle_path = os.path.join(out_dir, fname)

    # Stage dicts
    s1 = run.get("stage1", {})
    s3 = run.get("stage3", {})

    # Stage-1 metrics
    s1_hist_val   = list(map(float, s1.get("hist_val", [])))
    s1_best_val   = _safe_float(s1.get("best_val"), np.nan)
    s1_final_val  = _safe_last(s1_hist_val)
    s1_best_epoch = _safe_int(s1.get("best_epoch"), _safe_argmin(s1_hist_val))
    s1_epochs     = _safe_int(s1.get("epochs"), len(s1_hist_val))
    s1_seconds    = _safe_float(s1.get("train_seconds"), np.nan)

    # Stage-3 series
    s3_hist_val   = list(map(float, s3.get("hist_val", [])))
    s3_hist_noisy = list(map(float, s3.get("hist_noisy", [])))
    s3_hist_delta = list(map(float, s3.get("hist_delta", [])))

    s3_best_val   = _safe_float(s3.get("best_val"), np.nan)
    s3_final_val  = _safe_last(s3_hist_val)

    s3_best_epoch = _safe_int(s3.get("best_epoch"), _safe_argmin(s3_hist_val))
    s3_epochs     = _safe_int(s3.get("epochs"), len(s3_hist_val))
    s3_seconds    = _safe_float(s3.get("train_seconds"), np.nan)

    # --- compute metrics with FALLBACKS if curves are missing ---
    noisy_baseline = float(np.nanmean(s3_hist_noisy)) if len(s3_hist_noisy) else np.nan
    best_delta     = (float(np.nanmax(s3_hist_delta)) if (len(s3_hist_delta) and np.isfinite(np.nanmax(s3_hist_delta)))
                      else np.nan)
    final_delta    = _safe_last(s3_hist_delta)

    need_fallback = (not len(s3_hist_noisy)) or (not np.isfinite(noisy_baseline)) or (not np.isfinite(final_delta))

    if need_fallback:
        # Only attempt fallback if we have the necessary globals and arrays
        phi_for_L = np.array(s1.get("phi", []))
        psi_params = np.array(s3.get("psi", []))
        if phi_for_L.size and psi_params.size and ("stage3_qnodes" in globals()) and ("stage3_val_values_det" in globals()):
            # Rebuild the QNodes for this (L, phi) so we can evaluate psi on X_val
            stage3_handles = stage3_qnodes(L, phi_for_L)  # sets the fixed decoder from φ
            # X_val must exist for deterministic evaluation
            Xv = globals().get("X_val", None)
            if Xv is not None:
                mN, mD, d_pct = stage3_val_values_det(psi_params, Xv, sigma=globals().get("EVAL_SIGMA", 0.20))
                noisy_baseline = float(mN)
                final_delta    = float(d_pct)
                if not np.isfinite(best_delta):  # if we don't have a curve, use final as best
                    best_delta = final_delta

    # bundle JSON (parameters + training curves)
    bundle = {
        "schema": {"name": "half_qae_bundle", "version": "1.0"},
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "dataset": {
            "id": dataset_folder,
            "scale_low":  scale_low,
            "scale_high": scale_high,
            "window_size": int(nq),
            "window_stride": int(globals().get("WINDOW_STRIDE", 1)),
        },
        "run": {
            "tag": f"inst{inst}_L{L}",
            "instance_id": inst,
            "seed": seed,
            "sigma_train": float(globals().get("EVAL_SIGMA", np.nan)),
            "sigma_eval":  float(globals().get("EVAL_SIGMA", np.nan)),
        },
        "architecture": {
            "n_qubits": int(nq),
            "n_layers": int(L),
            "n_latent": int(nl),
            "n_trash":  int(nt),
            "latent_wires": list(range(int(nl))),
            "trash_wires":  list(range(int(nl), int(nq))),
        },
        "training": {
            "stage1": {
                "lr_init": S1_LR_INIT, "max_epochs": S1_MAX_EPOCHS,
                "patience": S1_PATIENCE, "lr_patience": S1_LR_PATIENCE,
                "best_val": s1_best_val, "final_val": s1_final_val,
                "best_epoch": s1_best_epoch, "epochs": s1_epochs,
                "train_curve": s1.get("hist_train", []), "val_curve": s1_hist_val, "lr_curve": s1.get("hist_lr", []),
                "train_seconds": s1_seconds,
            },
            "stage3": {
                "lr_init": S3_LR_INIT, "max_epochs": S3_MAX_EPOCHS,
                "patience": S3_PATIENCE, "plateau_steps": S3_PLATEAU_STEPS, "plateau_factor": S3_PLATEAU_FACT,
                "best_val_mse": s3_best_val, "final_val_mse": s3_final_val,
                "best_epoch": s3_best_epoch, "epochs": s3_epochs,
                "train_curve": s3.get("hist_train", []), "val_curve": s3_hist_val,
                "noisy_curve": s3.get("hist_noisy", []), "delta_curve": s3.get("hist_delta", []),
                "train_seconds": s3_seconds,
            }
        },
        "parameters": {
            "phi_stage1": np.array(s1.get("phi", [])).tolist(),
            "psi_stage3": np.array(s3.get("psi", [])).tolist(),
        },
    }
    with open(bundle_path, "w", encoding="utf-8") as f:
        json.dump(bundle, f, indent=2)
    print(f"Saved bundle → {bundle_path}")

    # assemble CSV row
    phi_params = json.dumps(bundle["parameters"]["phi_stage1"])
    psi_params = json.dumps(bundle["parameters"]["psi_stage3"])
    total_seconds = float((0 if np.isnan(s1_seconds) else s1_seconds) + (0 if np.isnan(s3_seconds) else s3_seconds))

    row = [
        os.path.basename(bundle_path),
        f"inst{inst}_L{L}",
        dataset_folder,
        inst, seed,
        int(nq), int(nl), int(nt), int(L),
        f"{globals().get('EVAL_SIGMA', np.nan):.3f}", f"{globals().get('EVAL_SIGMA', np.nan):.3f}", int(globals().get("WINDOW_STRIDE", 1)),
        f"{S1_LR_INIT:.6f}", int(S1_MAX_EPOCHS), int(S1_PATIENCE), int(S1_LR_PATIENCE),
        f"{s1_best_val:.8f}", f"{s1_final_val:.8f}", s1_best_epoch, s1_epochs, s1_seconds,
        f"{S3_LR_INIT:.6f}", int(S3_MAX_EPOCHS), int(S3_PATIENCE), int(S3_PLATEAU_STEPS), f"{S3_PLATEAU_FACT:.3f}",
        f"{s3_best_val:.8f}", f"{s3_final_val:.8f}", s3_best_epoch, s3_epochs, s3_seconds,
        noisy_baseline, best_delta, final_delta,
        phi_params, psi_params,
        total_seconds,
    ]

    # upsert row into CSV
    row_df = pd.DataFrame([row], columns=CSV_HEADER)
    if Path(CSV_PATH).exists():
        df_old = pd.read_csv(CSV_PATH)
        key = os.path.basename(bundle_path)
        if "filename" in df_old.columns:
            df_old = df_old[df_old["filename"] != key]
        df_new = pd.concat([df_old, row_df], ignore_index=True)
        df_new.to_csv(CSV_PATH, index=False)
    else:
        row_df.to_csv(CSV_PATH, index=False)
    print(f"Upserted CSV row  → {CSV_PATH}")

# ---- save all runs from Cell 7 ----
for run in RUNS:
    save_one_run(run)

print("\nAll runs saved and recorded.")

Saved bundle → ./runs_halfqae/q4_l2t2/L1/4q_2l_2t_1ls_01.json
Upserted CSV row  → ./runs_halfqae/all_training_instances_v2.csv
Saved bundle → ./runs_halfqae/q4_l2t2/L1/4q_2l_2t_1ls_02.json
Upserted CSV row  → ./runs_halfqae/all_training_instances_v2.csv
Saved bundle → ./runs_halfqae/q4_l2t2/L1/4q_2l_2t_1ls_03.json
Upserted CSV row  → ./runs_halfqae/all_training_instances_v2.csv
Saved bundle → ./runs_halfqae/q4_l2t2/L1/4q_2l_2t_1ls_04.json
Upserted CSV row  → ./runs_halfqae/all_training_instances_v2.csv
Saved bundle → ./runs_halfqae/q4_l2t2/L1/4q_2l_2t_1ls_05.json
Upserted CSV row  → ./runs_halfqae/all_training_instances_v2.csv
Saved bundle → ./runs_halfqae/q4_l2t2/L3/4q_2l_2t_3ls_01.json
Upserted CSV row  → ./runs_halfqae/all_training_instances_v2.csv
Saved bundle → ./runs_halfqae/q4_l2t2/L3/4q_2l_2t_3ls_02.json
Upserted CSV row  → ./runs_halfqae/all_training_instances_v2.csv
Saved bundle → ./runs_halfqae/q4_l2t2/L3/4q_2l_2t_3ls_03.json
Upserted CSV row  → ./runs_halfqae/all_training_i

In [11]:
# =========================================================
# Cell 9 — Build & preview the training-only results table
# =========================================================
import pandas as pd
import numpy as np
from pathlib import Path

if not Path(CSV_PATH).exists():
    raise FileNotFoundError(f"CSV not found: {CSV_PATH}. Run Cell 8 first.")

df = pd.read_csv(CSV_PATH)

# --- NEW: drop duplicate runs; keep the newest copy (with baseline/delta)
if "filename" in df.columns:
    df = df.drop_duplicates(subset=["filename"], keep="last")
else:
    df = df.drop_duplicates(subset=["run_tag","instance_id","n_layers"], keep="last")

# Typical numeric casts (safe)
for col in [
    "s3_noisy_baseline_mse","s3_best_delta_pct","s3_final_delta_pct",
    "s3_best_val_mse","s3_final_val_mse",
    "s1_best_val","s1_final_val",
    "s1_train_seconds","s3_train_seconds","total_train_seconds",
    "s1_best_epoch","s1_epochs","s3_best_epoch","s3_epochs"
]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.sort_values(["n_layers","instance_id"]).reset_index(drop=True)

clean_path = f"{OUT_BASE}/all_training_instances_{CSV_SCHEMA_VERSION}.csv"
Path(OUT_BASE).mkdir(parents=True, exist_ok=True)
df.to_csv(clean_path, index=False)
print(f"Saved training-only table → {clean_path}")

# A compact per-layer summary (mean±std); guards against all-NaN
def mean_std_safe(s: pd.Series) -> str:
    v = pd.to_numeric(s, errors="coerce").astype(float)
    v = v[np.isfinite(v)]
    if v.size == 0: return "n/a"
    return f"{v.mean():.6f} ± {v.std(ddof=0):.6f}"

metrics = [
    ("s3_noisy_baseline_mse", "noisy_baseline_mse (mean±std)"),
    ("s3_best_val_mse",       "best_val_mse (mean±std)"),
    ("s3_final_val_mse",      "final_val_mse (mean±std)"),
    ("s3_best_delta_pct",     "best_delta_pct (mean±std)"),
    ("s3_final_delta_pct",    "final_delta_pct (mean±std)"),
    ("s1_best_val",           "s1_best_val (mean±std)"),
]

grp = df.groupby("n_layers", dropna=False)
summary = pd.DataFrame({"runs": grp.size()})
for col, label in metrics:
    if col in df.columns and np.isfinite(df[col]).any():
        summary[label] = grp[col].apply(mean_std_safe)

summary_path = f"{OUT_BASE}/summary_by_layers_{CSV_SCHEMA_VERSION}.csv"
summary.to_csv(summary_path, index=True)
print(f"Saved per-layer summary → {summary_path}")

display(df.head(10))
display(summary)

Saved training-only table → ./runs_halfqae/all_training_instances_v2.csv
Saved per-layer summary → ./runs_halfqae/summary_by_layers_v2.csv


,filename,run_tag,dataset_folder,instance_id,rng_seed,n_qubits,n_latent,n_trash,n_layers,sigma_train,...,s3_final_val_mse,s3_best_epoch,s3_epochs,s3_train_seconds,s3_noisy_baseline_mse,s3_best_delta_pct,s3_final_delta_pct,phi_params,psi_params,total_train_seconds
0,4q_2l_2t_1ls_01.json,inst1_L1,mackey_glass_tau30_n200,1,1,4,2,2,1,0.2,...,0.009964,10,21,64.834131,0.013847,28.597628,28.038128,"[-0.016312803262711447, -1.986552627047903, 0....","[0.004653935659031972, -2.0253661313694864, -0...",115.960609
1,4q_2l_2t_1ls_02.json,inst2_L1,mackey_glass_tau30_n200,2,2,4,2,2,1,0.2,...,0.009799,12,23,72.580082,0.013847,30.144129,29.231419,"[0.011801558810262056, -1.9347410256049633, 0....","[0.02414285247762957, -2.0094330925634307, -0....",127.678969
2,4q_2l_2t_1ls_03.json,inst3_L1,mackey_glass_tau30_n200,3,3,4,2,2,1,0.2,...,0.009744,27,38,116.465180,0.013847,29.835796,29.629363,"[0.00028094502693164604, 1.096912804007125, 0....","[0.015007652271541886, 1.0230249899218355, 1.3...",156.282630
3,4q_2l_2t_1ls_04.json,inst4_L1,mackey_glass_tau30_n200,4,4,4,2,2,1,0.2,...,0.009898,23,34,105.541854,0.013847,28.856028,28.519410,"[0.03214672941676667, -2.0208978324444513, -0....","[-0.019716129742746204, -2.1269129615096083, 0...",155.306374
4,4q_2l_2t_1ls_05.json,inst5_L1,mackey_glass_tau30_n200,5,5,4,2,2,1,0.2,...,0.009874,21,32,103.156757,0.013847,29.760033,28.689862,"[0.00033290752957749667, -1.9049135233050598, ...","[0.029558322431367485, -2.0536424211010313, -0...",150.174163
5,4q_2l_2t_3ls_01.json,inst1_L3,mackey_glass_tau30_n200,1,1,4,2,2,3,0.2,...,0.009519,12,23,177.698407,0.013847,32.522158,31.250727,"[1.2278353250797251, -0.029796534519642336, -0...","[1.3993129974539795, -0.02034789510104483, -0....",336.080375
6,4q_2l_2t_3ls_02.json,inst2_L3,mackey_glass_tau30_n200,2,2,4,2,2,3,0.2,...,0.009066,16,27,219.724816,0.013847,35.453481,34.523729,"[-0.0043693490827218075, -0.05240699121450067,...","[-0.004682019395181499, 0.011726924508950902, ...",385.865487
7,4q_2l_2t_3ls_03.json,inst3_L3,mackey_glass_tau30_n200,3,3,4,2,2,3,0.2,...,0.010089,20,31,249.047306,0.013847,27.629534,27.134850,"[0.7862941374481595, 1.4073252568117587, 0.780...","[0.8407786622971607, 1.5477164981938103, 0.841...",491.122231
8,4q_2l_2t_3ls_04.json,inst4_L3,mackey_glass_tau30_n200,4,4,4,2,2,3,0.2,...,0.009260,24,35,277.540700,0.013847,33.191473,33.121094,"[0.007009495854836537, -0.027214479629045413, ...","[0.038188889968790274, -0.05859270351253336, 1...",503.708784
9,4q_2l_2t_3ls_05.json,inst5_L3,mackey_glass_tau30_n200,5,5,4,2,2,3,0.2,...,0.009853,6,17,135.293316,0.013847,29.264122,28.840395,"[1.5714525489700095, -0.008458921277344641, 1....","[1.5928106508978992, -0.0319637609603955, 1.46...",368.880210


,runs,noisy_baseline_mse (mean±std),best_val_mse (mean±std),final_val_mse (mean±std),best_delta_pct (mean±std),final_delta_pct (mean±std),s1_best_val (mean±std)
n_layers,,,,,,,
1,5,0.013847 ± 0.000000,0.009770 ± 0.000083,0.009856 ± 0.000077,29.438723 ± 0.600913,28.821636 ± 0.555506,0.268295 ± 0.004261
3,5,0.013847 ± 0.000000,0.009469 ± 0.000389,0.009558 ± 0.000375,31.612154 ± 2.809009,30.974159 ± 2.704756,0.081151 ± 0.020465
